In [1]:
from pymol import cmd
import py3Dmol
import pandas as pd
import random
from openbabel import pybel
import networkx as nx
#from pyvis import network as pnet
from matplotlib import cm, colors
from matplotlib import pyplot as plt
from IPython.display import IFrame
import math
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

import sys, os, random
sys.path.insert(1, 'utilities/')

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import py3Dmol
from rdkit.Geometry import Point3D
from MDAnalysis import Merge

import prolif as plf
#from prolif_old.plotting.network import LigNetwork

from utils import getbox, generate_ledock_file, dok_to_sdf,fix_protein

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False



RDKit WARNING: [23:13:58] Enabling RDKit 2019.09.3 jupyter extensions
[23:13:58] Enabling RDKit 2019.09.3 jupyter extensions


Specify Inputs

In [2]:
#The relative working folder path to use
Receptor_Folder='Olaparib/PARP2/' 

#The PDB ID to fetch and prepare
pdbID='4TVJ' 


Fetch and prepare protien from the PDB (skip if files already exist in the project folder)

Give permissions: chmod u+x bin/lepro_linux_x86

In [3]:
#Change the directory
os.chdir(Receptor_Folder)

#Fetch protien and native ligand from the pdb 
cmd.fetch(code=pdbID,type='pdb1')
cmd.select(name='Prot',selection='polymer.protein')
cmd.select(name='NativeLigand',selection='organic')
cmd.save(filename=str(pdbID)+'_clean.pdb',format='pdb',selection='Prot')
cmd.save(filename=str(pdbID)+'_NativeLigand.mol2',format='mol2',selection='NativeLigand')
cmd.delete('all')

#Create cleaned receptor file using lepro
lepro_input=str(pdbID)+'_clean.pdb'
!../../bin/lepro_linux_x86 {lepro_input}
os.rename('pro.pdb',str(pdbID)+'_clean_H.pdb') 

 PyMOL not running, entering library mode (experimental)


Process from protien and ligand using existing files

In [3]:
#Change the directory
os.chdir(Receptor_Folder)

protien_file="4TVJ_clean.pdb"
native_ligand_file="4TVJ_ligand.pdb"
#Fetch protien and native ligand from the pdb 
cmd.load(protien_file)
cmd.load(native_ligand_file)
cmd.select(name='Prot',selection='polymer.protein')
cmd.select(name='NativeLigand',selection='organic')
cmd.save(filename=str(pdbID)+'_clean_lepro.pdb',format='pdb',selection='Prot')
cmd.save(filename=str(pdbID)+'_NativeLigand.mol2',format='mol2',selection='NativeLigand')
cmd.delete('all')

#Create cleaned receptor file using lepro
lepro_input=str(pdbID)+'_clean_lepro.pdb'
!../../bin/lepro_linux_x86 {lepro_input}
os.rename('pro.pdb',str(pdbID)+'_clean_H.pdb') 

 PyMOL not running, entering library mode (experimental)


Calculate gridbox coords based on the specified ligand

In [4]:
cmd.load(filename=str(pdbID)+'_clean_H.pdb',format='pdb',object='prot') #Not needed but as reference of the system
cmd.load(filename=str(pdbID)+'_NativeLigand.mol2',format='mol2',object='lig')
center,size=getbox(selection='lig',extending=5.0,software='vina')
cmd.delete('all')

print('Center:', center)
print('Size:', size)

Center: {'center_x': 18.833499908447266, 'center_y': -2.2054998874664307, 'center_z': 17.501500129699707}
Size: {'size_x': 17.152999877929688, 'size_y': 20.658999919891357, 'size_z': 19.641000747680664}
